In [ ]:
# 데이터 분석 기본 라이브러리 관련
import os
import sys
import platform
import warnings
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook

# EDA 관련
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#import missingno as msno

# 데이터 전처리 관련
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from matplotlib.dates import YearLocator, MonthLocator
import matplotlib.cm as cm

In [ ]:
warnings.filterwarnings(action='ignore')

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 200
plt.style.use('ggplot')

current_os = platform.platform().split("-")[0]
if current_os == 'Windows':
    print(f'현재 사용되는 운영체제: {current_os}')
    plt.rc('font', family='Malgun Gothic') # For Windows
    plt.rc("axes",unicode_minus=False)
elif current_os == 'Darwin': # macOS
    print(f"본 내용은 Windows 환경에서 분석됐습니다. 다른 OS로 하시게 될 경우 에러가 날 수 있습니다.")
    plt.rc('font', family='AppleGothic') # For Windows

현재 사용되는 운영체제: Windows


In [ ]:
삼성전자=pd.read_csv('C:/Users/sas/Desktop/big data analysis/data/삼성전자.csv',encoding='cp949')
SK하이닉스=pd.read_csv('C:/Users/sas/Desktop/big data analysis/data/SK하이닉스.csv',encoding='cp949')
NAVER=pd.read_csv('C:/Users/sas/Desktop/big data analysis/data/NAVER.csv',encoding='cp949')
씨젠=pd.read_csv('C:/Users/sas/Desktop/big data analysis/data/씨젠.csv',encoding='cp949')
우리들휴브레인=pd.read_csv('C:/Users/sas/Desktop/big data analysis/data/우리들휴브레인.csv',encoding='cp949')
현대차=pd.read_csv('C:/Users/sas/Desktop/big data analysis/data/현대차.csv',encoding='cp949')
DGB금융지주=pd.read_csv('C:/Users/sas/Desktop/big data analysis/data/DGB금융지주.csv',encoding='cp949')
미스터블루=pd.read_csv('C:/Users/sas/Desktop/big data analysis/data/미스터블루.csv',encoding='cp949')
셀트리온=pd.read_csv('C:/Users/sas/Desktop/big data analysis/data/셀트리온.csv',encoding='cp949')
데일리블록체인=pd.read_csv('C:/Users/sas/Desktop/big data analysis/data/데일리블록체인.csv',encoding='cp949')
소리바다=pd.read_csv('C:/Users/sas/Desktop/big data analysis/data/소리바다.csv',encoding='cp949')
한화솔루션=pd.read_csv('C:/Users/sas/Desktop/big data analysis/data/한화솔루션.csv',encoding='cp949')
아모레퍼시픽=pd.read_csv('C:/Users/sas/Desktop/big data analysis/data/아모레퍼시픽.csv',encoding='cp949')
CJ대한통운=pd.read_csv('C:/Users/sas/Desktop/big data analysis/data/CJ대한통운.csv',encoding='cp949')
GS건설=pd.read_csv('C:/Users/sas/Desktop/big data analysis/data/GS건설.csv',encoding='cp949')

___

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

In [ ]:
from sklearn import neighbors
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

In [ ]:
from fbprophet import Prophet
import math

In [ ]:
def pred(stock):
    ######### data set 만들기
    stock ['날짜']=pd.to_datetime(stock.날짜,format='%Y-%m-%d')
    stock.index=stock['날짜']
    stock=stock[(stock.index>='2013.01.01')&(stock.index<='2019.12.31')]
    data=stock[['종가']]
    
    ####moving average
    train=data[(data.index>='2013-01-01') & (data.index<'2019-09-01')] 
    test=data[(data.index>='2019-09-01') & (data.index<='2019-12-31')]

    preds=[]
    for i in range(0,test.shape[0]):
        a=train['종가'][len(train)-82+i:].sum()+sum(preds)
        b=a/82
        preds.append(b)
        
    test["MA"] = preds
    MA = test
    
    #####Support Vector Regression ( RBF Kernel )
    data=stock[['날짜','종가']]
    data.reset_index(drop=True,inplace=True)
    train=data[(data.날짜>='2013-01-01') & (data.날짜<'2019-09-01')]
    test=data[(data.날짜>='2019-09-01') & (data.날짜<='2019-12-31')]
    
    x=data.drop('종가',1)
    y=data['종가']
    
    x_scaled=sc.fit_transform(x)
    x_scaled=pd.DataFrame(x_scaled,index=x.index)
    tr_x, te_x=train_test_split(x_scaled,test_size=81,shuffle=False)
    
    new_y=data['종가'].values.reshape(-1,1)

    x_scaled=sc.fit_transform(x)
    x_scaled=pd.DataFrame(x_scaled,index=x.index)
    y_scaled=sc.fit_transform(new_y)
    y_scaled=pd.DataFrame(y_scaled,index=y.index)
    tr_x2, te_x2=train_test_split(x_scaled,test_size=81,shuffle=False)
    tr_y2, te_y2=train_test_split(y_scaled,test_size=81,shuffle=False)

    svc1=SVR(kernel='rbf')
    svc1.fit(tr_x2,tr_y2)

    svc_pred_rbf=svc1.predict(te_x)

    y_pred_svc_rbf=sc.inverse_transform(svc_pred_rbf)
    
    MA['RBF']=y_pred_svc_rbf 
    
    ####prophet
    data=stock[['날짜','종가']]
    data.reset_index(drop=True,inplace=True)
    train=data[(data.날짜>='2013-01-01') & (data.날짜<'2019-09-01')]
    test=data[(data.날짜>='2019-09-01') & (data.날짜<='2019-12-31')]
    
    df = train.reset_index().rename(columns={'날짜':'ds', '종가':'y'})
    df['y'] = np.log(df['y'])
    model = Prophet()
    model.fit(df);
    future = model.make_future_dataframe(periods=81) 
    forecast = model.predict(future)
    
    yhat = forecast.set_index('ds').join(train)

    a = np.array(yhat["yhat"].values.tolist())
    yhatt = np.exp(a)
    yhat = yhatt[-81:]
    
    MA['prophet'] = yhat

    return MA

---

In [ ]:
삼성전자 = pred(삼성전자)
삼성전자.to_csv("C:\\Users\\sas\\Desktop\\big data analysis\\pred\\삼성전자.csv",header=True, index=True)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
SK하이닉스 = pred(SK하이닉스)
SK하이닉스.to_csv("C:\\Users\\sas\\Desktop\\big data analysis\\pred\\SK하이닉스.csv",header=True, index=True)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
GS건설 = pred(GS건설)
GS건설.to_csv("C:\\Users\\sas\\Desktop\\big data analysis\\pred\\GS건설.csv",header=True, index=True)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
현대차 = pred(현대차)
현대차.to_csv("C:\\Users\\sas\\Desktop\\big data analysis\\pred\\현대차.csv",header=True, index=True)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
NAVER = pred(NAVER)
NAVER.to_csv("C:\\Users\\sas\\Desktop\\big data analysis\\pred\\NAVER.csv",header=True, index=True)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
DGB금융지주 = pred(DGB금융지주)
DGB금융지주.to_csv("C:\\Users\\sas\\Desktop\\big data analysis\\pred\\DGB금융지주.csv",header=True, index=True)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
CJ대한통운 = pred(CJ대한통운)
CJ대한통운.to_csv("C:\\Users\\sas\\Desktop\\big data analysis\\pred\\CJ대한통운.csv",header=True, index=True)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
아모레퍼시픽 = pred(아모레퍼시픽)
아모레퍼시픽.to_csv("C:\\Users\\sas\\Desktop\\big data analysis\\pred\\아모레퍼시픽.csv",header=True, index=True)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
씨젠 = pred(씨젠)
씨젠.to_csv("C:\\Users\\sas\\Desktop\\big data analysis\\pred\\씨젠.csv",header=True, index=True)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
셀트리온 = pred(셀트리온)
셀트리온.to_csv("C:\\Users\\sas\\Desktop\\big data analysis\\pred\\셀트리온.csv",header=True, index=True)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
소리바다 = pred(소리바다)
소리바다.to_csv("C:\\Users\\sas\\Desktop\\big data analysis\\pred\\소리바다.csv",header=True, index=True)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
데일리블록체인 = pred(데일리블록체인)
데일리블록체인.to_csv("C:\\Users\\sas\\Desktop\\big data analysis\\pred\\데일리블록체인.csv",header=True, index=True)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
미스터블루 = pred(미스터블루)
미스터블루.to_csv("C:\\Users\\sas\\Desktop\\big data analysis\\pred\\미스터블루.csv",header=True, index=True)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
우리들휴브레인 = pred(우리들휴브레인)
우리들휴브레인.to_csv("C:\\Users\\sas\\Desktop\\big data analysis\\pred\\우리들휴브레인.csv",header=True, index=True)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
한화솔루션 = pred(한화솔루션)
한화솔루션.to_csv("C:\\Users\\sas\\Desktop\\big data analysis\\pred\\한화솔루션.csv",header=True, index=True)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
